# minibpe exercise
Exercise as suggested from Karpathy on [minibpe/exercise](https://github.com/karpathy/minbpe/blob/master/exercise.md).

## Step 1- Train only BasicTokenizer

In [4]:
class BasicTokenizer:
  def __init__(self):
      self.merges = {}
      self.tokens = []
      self.vocab = {idx: bytes([idx]) for idx in range(256)}

  def get_stats(self, ids):
      counts = {}
      for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
      return counts

  def merge(self, ids, pair, idx):
      newids = []
      i = 0
      while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
          newids.append(idx)
          i += 2
        else:
          newids.append(ids[i])
          i += 1
      return newids

  def train(self, text, vocab_size, verbose=False):
      original_byte_size = 256
      num_merges = vocab_size - original_byte_size

      self.tokens = text.encode("utf-8") # raw bytes
      self.tokens = list(map(int, self.tokens)) # convert to a list of integers in range 0..255 for convenience
      ids = list(self.tokens)

      for i in range(num_merges):
        stats = self.get_stats(ids)
        pair = max(stats, key=stats.get)
        idx = 256 + i
        if (verbose):
          print(f"merging {pair} into a new token {idx}")
        ids = self.merge(ids, pair, idx)
        self.merges[pair] = idx

      for (p0, p1), idx in self.merges.items():
        self.vocab[idx] = self.vocab[p0] + self.vocab[p1]

  def encode(self, text):
    tokens = list(text.encode("utf-8"))
    while len(tokens) >= 2:
      stats = self.get_stats(tokens)
      pair = min(stats, key=lambda p: self.merges.get(p, float("inf")))
      if pair not in self.merges:
        break # nothing else can be merged
      idx = self.merges[pair]
      tokens = self.merge(tokens, pair, idx)

    return tokens

  def decode(self, ids):
    tokens = b"".join(self.vocab[idx] for idx in ids)
    text = tokens.decode("utf-8", errors="replace")
    
    return text

In [5]:
# open wiki_ww2.txt and read whole file
with open("wiki_ww2.txt", "r") as file:
  text = file.read()

In [6]:
tokenizer = BasicTokenizer()
tokenizer.train(text, 1000, verbose=True)

merging (101, 32) into a new token 256
merging (97, 110) into a new token 257
merging (100, 32) into a new token 258
merging (105, 110) into a new token 259
merging (116, 104) into a new token 260
merging (101, 114) into a new token 261
merging (46, 32) into a new token 262
merging (44, 32) into a new token 263
merging (115, 32) into a new token 264
merging (111, 110) into a new token 265
merging (260, 256) into a new token 266
merging (97, 114) into a new token 267
merging (111, 114) into a new token 268
merging (121, 32) into a new token 269
merging (116, 32) into a new token 270
merging (101, 258) into a new token 271
merging (97, 108) into a new token 272
merging (32, 32) into a new token 273
merging (97, 116) into a new token 274
merging (101, 110) into a new token 275
merging (101, 115) into a new token 276
merging (49, 57) into a new token 277
merging (111, 102) into a new token 278
merging (32, 266) into a new token 279
merging (257, 258) into a new token 280
merging (105, 116)

In [12]:
# visualize the merged tokens
for (p0, p1), idx in tokenizer.merges.items():
  print(tokenizer.vocab[p0], "with", tokenizer.vocab[p1], "=>", tokenizer.vocab[idx])

b'e' with b' ' => b'e '
b'a' with b'n' => b'an'
b'd' with b' ' => b'd '
b'i' with b'n' => b'in'
b't' with b'h' => b'th'
b'e' with b'r' => b'er'
b'.' with b' ' => b'. '
b',' with b' ' => b', '
b's' with b' ' => b's '
b'o' with b'n' => b'on'
b'th' with b'e ' => b'the '
b'a' with b'r' => b'ar'
b'o' with b'r' => b'or'
b'y' with b' ' => b'y '
b't' with b' ' => b't '
b'e' with b'd ' => b'ed '
b'a' with b'l' => b'al'
b' ' with b' ' => b'  '
b'a' with b't' => b'at'
b'e' with b'n' => b'en'
b'e' with b's' => b'es'
b'1' with b'9' => b'19'
b'o' with b'f' => b'of'
b' ' with b'the ' => b' the '
b'an' with b'd ' => b'and '
b'i' with b't' => b'it'
b'i' with b'c' => b'ic'
b'i' with b'on' => b'ion'
b'2' with b'0' => b'20'
b'in' with b'g' => b'ing'
b'.' with b'\n' => b'.\n'
b'i' with b's' => b'is'
b'r' with b'o' => b'ro'
b'of' with b' ' => b'of '
b'an' with b' ' => b'an '
b'r' with b'e' => b're'
b'i' with b'e' => b'ie'
b't' with b'o' => b'to'
b'o' with b'l' => b'ol'
b':' with b' ' => b': '
b'er' with b' 

In [27]:
tokenizer.vocab

{0: b'\x00',
 1: b'\x01',
 2: b'\x02',
 3: b'\x03',
 4: b'\x04',
 5: b'\x05',
 6: b'\x06',
 7: b'\x07',
 8: b'\x08',
 9: b'\t',
 10: b'\n',
 11: b'\x0b',
 12: b'\x0c',
 13: b'\r',
 14: b'\x0e',
 15: b'\x0f',
 16: b'\x10',
 17: b'\x11',
 18: b'\x12',
 19: b'\x13',
 20: b'\x14',
 21: b'\x15',
 22: b'\x16',
 23: b'\x17',
 24: b'\x18',
 25: b'\x19',
 26: b'\x1a',
 27: b'\x1b',
 28: b'\x1c',
 29: b'\x1d',
 30: b'\x1e',
 31: b'\x1f',
 32: b' ',
 33: b'!',
 34: b'"',
 35: b'#',
 36: b'$',
 37: b'%',
 38: b'&',
 39: b"'",
 40: b'(',
 41: b')',
 42: b'*',
 43: b'+',
 44: b',',
 45: b'-',
 46: b'.',
 47: b'/',
 48: b'0',
 49: b'1',
 50: b'2',
 51: b'3',
 52: b'4',
 53: b'5',
 54: b'6',
 55: b'7',
 56: b'8',
 57: b'9',
 58: b':',
 59: b';',
 60: b'<',
 61: b'=',
 62: b'>',
 63: b'?',
 64: b'@',
 65: b'A',
 66: b'B',
 67: b'C',
 68: b'D',
 69: b'E',
 70: b'F',
 71: b'G',
 72: b'H',
 73: b'I',
 74: b'J',
 75: b'K',
 76: b'L',
 77: b'M',
 78: b'N',
 79: b'O',
 80: b'P',
 81: b'Q',
 82: b'R',
 83: b'

## Step 2- RegexTokenizer

In [13]:
GPT4_SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""

In [16]:
import regex as re

In [17]:
split_text = re.findall(GPT4_SPLIT_PATTERN, text)
split_text

['\n',
 'Main',
 ' menu',
 '\n\n',
 'Wikipedia',
 ' The',
 ' Free',
 ' Encyclopedia',
 '\n\n',
 '   ',
 ' Create',
 ' account',
 '\n',
 '   ',
 ' Log',
 ' in',
 '\n\n',
 'Personal',
 ' tools',
 '\n\n',
 'Contents',
 '\n',
 '(Top',
 ')\n',
 'Start',
 ' and',
 ' end',
 ' dates',
 '\n\n',
 'History',
 '\n\n',
 'Aftermath',
 '\n\n',
 'Impact',
 '\n\n',
 '   ',
 ' See',
 ' also',
 '\n',
 '   ',
 ' Notes',
 '\n',
 '   ',
 ' Citations',
 '\n',
 '   ',
 ' References',
 '\n',
 '   ',
 ' External',
 ' links',
 '\n\n',
 'World',
 ' War',
 ' II',
 '\n\n',
 '   ',
 ' Article',
 '\n',
 '   ',
 ' Talk',
 '\n\n',
 '   ',
 ' Read',
 '\n',
 '   ',
 ' View',
 ' source',
 '\n',
 '   ',
 ' View',
 ' history',
 '\n\n',
 'Tools',
 '\n\n',
 'Appearance',
 '\n',
 'Text',
 '\n\n',
 '   ',
 ' Small',
 '\n',
 '   ',
 ' Standard',
 '\n',
 '   ',
 ' Large',
 '\n\n',
 'Width',
 '\n\n',
 '   ',
 ' Standard',
 '\n',
 '   ',
 ' Wide',
 '\n\n',
 'Color',
 ' (',
 'beta',
 ')\n\n',
 '   ',
 ' Automatic',
 '\n',
 '   ',
 '

In [29]:
[[int(byte) for byte in text.encode("utf-8")] for text in split_text]

[[10],
 [77, 97, 105, 110],
 [32, 109, 101, 110, 117],
 [10, 10],
 [87, 105, 107, 105, 112, 101, 100, 105, 97],
 [32, 84, 104, 101],
 [32, 70, 114, 101, 101],
 [32, 69, 110, 99, 121, 99, 108, 111, 112, 101, 100, 105, 97],
 [10, 10],
 [32, 32, 32],
 [32, 67, 114, 101, 97, 116, 101],
 [32, 97, 99, 99, 111, 117, 110, 116],
 [10],
 [32, 32, 32],
 [32, 76, 111, 103],
 [32, 105, 110],
 [10, 10],
 [80, 101, 114, 115, 111, 110, 97, 108],
 [32, 116, 111, 111, 108, 115],
 [10, 10],
 [67, 111, 110, 116, 101, 110, 116, 115],
 [10],
 [40, 84, 111, 112],
 [41, 10],
 [83, 116, 97, 114, 116],
 [32, 97, 110, 100],
 [32, 101, 110, 100],
 [32, 100, 97, 116, 101, 115],
 [10, 10],
 [72, 105, 115, 116, 111, 114, 121],
 [10, 10],
 [65, 102, 116, 101, 114, 109, 97, 116, 104],
 [10, 10],
 [73, 109, 112, 97, 99, 116],
 [10, 10],
 [32, 32, 32],
 [32, 83, 101, 101],
 [32, 97, 108, 115, 111],
 [10],
 [32, 32, 32],
 [32, 78, 111, 116, 101, 115],
 [10],
 [32, 32, 32],
 [32, 67, 105, 116, 97, 116, 105, 111, 110, 115]

In [30]:
class RegexTokenizer:
  def __init__(self):
      self.merges = {}
      self.tokens = []
      self.vocab = {idx: bytes([idx]) for idx in range(256)}
      self.GPT4_SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""

  def get_stats(self, ids):
      counts = {}
      for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
      return counts

  def merge(self, ids, pair, idx):
      newids = []
      i = 0
      while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
          newids.append(idx)
          i += 2
        else:
          newids.append(ids[i])
          i += 1
      return newids

  def train(self, text, vocab_size, verbose=False):
    original_byte_size = 256
    num_merges = vocab_size - original_byte_size
    split_text = re.findall(self.GPT4_SPLIT_PATTERN, text)
    encoded_split_text = [[int(byte) for byte in text.encode("utf-8")] for text in split_text]
    
    # Initialize vocabulary with byte values
    self.vocab = {i: bytes([i]).decode('utf-8', errors='replace') for i in range(original_byte_size)}
    
    for i in range(num_merges):
        all_stats = {}
        for sublist in encoded_split_text:
            stats = self.get_stats(sublist)
            for pair, count in stats.items():
                all_stats[pair] = all_stats.get(pair, 0) + count
        
        if not all_stats:
            break
        
        pair = max(all_stats, key=all_stats.get)
        idx = 256 + i
        if verbose:
            print(f"merging {pair} into a new token {idx}")
        
        encoded_split_text = [self.merge(sublist, pair, idx) for sublist in encoded_split_text]
        self.merges[pair] = idx
        self.vocab[idx] = self.vocab[pair[0]] + self.vocab[pair[1]]
    
    self.tokens = [token for sublist in encoded_split_text for token in sublist]

  def encode(self, text):
    tokens = list(text.encode("utf-8"))
    while len(tokens) >= 2:
      stats = self.get_stats(tokens)
      pair = min(stats, key=lambda p: self.merges.get(p, float("inf")))
      if pair not in self.merges:
        break # nothing else can be merged
      idx = self.merges[pair]
      tokens = self.merge(tokens, pair, idx)

    return tokens

  def decode(self, ids):
    tokens = b"".join(self.vocab[idx] for idx in ids)
    text = tokens.decode("utf-8", errors="replace")
    
    return text

In [31]:
regex_tokenizer = RegexTokenizer()
regex_tokenizer.train(text, 1000, verbose=True)

merging (97, 110) into a new token 256
merging (32, 116) into a new token 257
merging (105, 110) into a new token 258
merging (104, 101) into a new token 259
merging (101, 114) into a new token 260
merging (111, 110) into a new token 261
merging (257, 259) into a new token 262
merging (97, 114) into a new token 263
merging (101, 100) into a new token 264
merging (101, 115) into a new token 265
merging (111, 114) into a new token 266
merging (97, 116) into a new token 267
merging (32, 111) into a new token 268
merging (97, 108) into a new token 269
merging (105, 116) into a new token 270
merging (256, 100) into a new token 271
merging (101, 110) into a new token 272
merging (32, 65) into a new token 273
merging (32, 112) into a new token 274
merging (49, 57) into a new token 275
merging (115, 116) into a new token 276
merging (268, 102) into a new token 277
merging (32, 258) into a new token 278
merging (32, 271) into a new token 279
merging (32, 83) into a new token 280
merging (105, 9

In [32]:
for (p0, p1), idx in regex_tokenizer.merges.items():
  print(regex_tokenizer.vocab[p0], "with", regex_tokenizer.vocab[p1], "=>", regex_tokenizer.vocab[idx])

a with n => an
  with t =>  t
i with n => in
h with e => he
e with r => er
o with n => on
 t with he =>  the
a with r => ar
e with d => ed
e with s => es
o with r => or
a with t => at
  with o =>  o
a with l => al
i with t => it
an with d => and
e with n => en
  with A =>  A
  with p =>  p
1 with 9 => 19
s with t => st
 o with f =>  of
  with in =>  in
  with and =>  and
  with S =>  S
i with c => ic
i with on => ion
  with w =>  w
in with g => ing
2 with 0 => 20
  with I =>  I
  with   =>   
. with 
 => .

  with W =>  W
  with f =>  f
  with a =>  a
r with o => ro
  with C =>  C
r with e => re
  with P =>  P
  with ( =>  (
  with c =>  c
i with s => is
o with l => ol
  with M =>  M
  with b =>  b
e with t => et
i with l => il
  with T =>  T
  with J =>  J
  with G =>  G
  with s =>  s
� with � => ��
  with R =>  R
  with d =>  d
 t with o =>  to
  with B =>  B
m with an => man
i with v => iv
a with s => as
20 with 0 => 200
a with p => ap
�� with � => ���
u with r => ur
) with . => ).

In [33]:
regex_tokenizer.vocab

{0: '\x00',
 1: '\x01',
 2: '\x02',
 3: '\x03',
 4: '\x04',
 5: '\x05',
 6: '\x06',
 7: '\x07',
 8: '\x08',
 9: '\t',
 10: '\n',
 11: '\x0b',
 12: '\x0c',
 13: '\r',
 14: '\x0e',
 15: '\x0f',
 16: '\x10',
 17: '\x11',
 18: '\x12',
 19: '\x13',
 20: '\x14',
 21: '\x15',
 22: '\x16',
 23: '\x17',
 24: '\x18',
 25: '\x19',
 26: '\x1a',
 27: '\x1b',
 28: '\x1c',
 29: '\x1d',
 30: '\x1e',
 31: '\x1f',
 32: ' ',
 33: '!',
 34: '"',
 35: '#',
 36: '$',
 37: '%',
 38: '&',
 39: "'",
 40: '(',
 41: ')',
 42: '*',
 43: '+',
 44: ',',
 45: '-',
 46: '.',
 47: '/',
 48: '0',
 49: '1',
 50: '2',
 51: '3',
 52: '4',
 53: '5',
 54: '6',
 55: '7',
 56: '8',
 57: '9',
 58: ':',
 59: ';',
 60: '<',
 61: '=',
 62: '>',
 63: '?',
 64: '@',
 65: 'A',
 66: 'B',
 67: 'C',
 68: 'D',
 69: 'E',
 70: 'F',
 71: 'G',
 72: 'H',
 73: 'I',
 74: 'J',
 75: 'K',
 76: 'L',
 77: 'M',
 78: 'N',
 79: 'O',
 80: 'P',
 81: 'Q',
 82: 'R',
 83: 'S',
 84: 'T',
 85: 'U',
 86: 'V',
 87: 'W',
 88: 'X',
 89: 'Y',
 90: 'Z',
 91: '[',


Indeed, we now see that we have no tokens that go across categories (numbers, letters, punctuation, more than one whitespace).